In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import networkx as nx
import stp_parser as stparser
from pyomo.environ import (
    Binary,
    ConcreteModel,
    Constraint,
    ConstraintList,
    Objective,
    Param,
    Set,
    SolverFactory,
    Var,
    minimize,
)

In [3]:
%matplotlib inline

# Init

In [4]:
ENV_PATH = Path(os.environ.get("CONDA_PREFIX"))
SOLVER_DIR = ENV_PATH / "envs/landscape_steiner/bin"  # set your path to the env

# Optimization Model

TODO List

- [ ] add cut constraints

In [5]:
class MPPCST:
    """Multi-period Prize Collecting Steiner Tree Problem.
    
    Ref: https://chairelogistique.hec.ca/wp-content/uploads/2020/09/Steiner.pdf
    
    TODO: 
        * refactor var/arg v0
        * add cut constraints
    """
    def __init__(
        self,
        stp: stparser.SteinerTreeProblem,
        verbose: bool=False,
        v0: int=None,  # TODO: refactor
        update_factor: dict={1: 0.5, 2: 1.0},  # TODO: refactor
        distance_limit: dict={1: 20, 2: 10},  # TODO: refactor
    ):
        self.verbose = verbose
        self.create_relaxed_model(v0, update_factor, distance_limit)
    
    def create_relaxed_model(self, v0, update_factor, distance_limit):  # TODO: refactor
        m = ConcreteModel(doc=("MPPCST"
            " Ref: [Faria2020]:"
            " https://chairelogistique.hec.ca/wp-content/uploads/2020/09/Steiner.pdf"))
        
        edge_set = list(stp.edge_attributes_df().index.unique())
        node_set = list(stp.node_attributes_df().index.unique())

        m.EDGES = Set(initialize=edge_set, dimen=2)
        m.NODES = Set(initialize=node_set)
        m.PERIODS = Set(initialize=list(stp.iter_periods()))
        
        m.v0 = Param(initialize=v0, doc="original network contracted in one node")
        m.update_factor = Param(m.PERIODS, initialize=update_factor)
        m.prize = Param(m.NODES, initialize=stp.node_attributes_df()["prize"].to_dict())
        m.distance_limit = Param(m.PERIODS, initialize=distance_limit)  # TODO: refactor
        
        edge_weight_dict = {(*k, t): v 
                            for k,v in stp.edge_attributes_df()["weight"].to_dict().items() 
                            for t in set(stp.iter_periods())}
        m.edge_weight = Param(m.EDGES * m.PERIODS, initialize=edge_weight_dict)
        m.edge_distance = Param(m.EDGES, initialize=stp.edge_attributes_df()["distance"].to_dict())
        
        m.YT = Var(m.NODES * m.PERIODS, domain=Binary)
        m.XT = Var(m.EDGES * m.PERIODS, domain=Binary)
        
        m.cutConstraints = ConstraintList(doc="Faria2020 eq 5")

        def multiPeriodConstraintX_rule(m, e):
            return sum(m.XT[e,t] for e in m.EDGES for t in m.PERIODS) <= 1
        m.multiPeriodConstraintX = Constraint(m.EDGES, doc="Faria2020 eq 6")
        
        def multiPeriodConstraintY_rule(m, e):
            return sum(m.YT[n,t] for n in m.NODES for t in m.PERIODS) <= 1
        m.multiPeriodConstraintY = Constraint(m.NODES, doc="Faria2018 eq 7")
        
        def distance_limit_rule(m, t):
            return sum(m.edge_distance[e] * m.XT[e,t] 
                       for e in m.EDGES 
                       for t in m.PERIODS) <= m.distance_limit[t]
        m.distanceLimitConstraint = Constraint(m.PERIODS, 
                                               rule=distance_limit_rule, 
                                               doc="Faria2020 eq 9")

        def connectivityConstraint_rule(m, t):
            return (sum(m.XT[e, tp] 
                        for e in m.EDGES 
                        for tp in m.PERIODS if tp <= t) 
            >=
            sum(m.YT[n, tp] for n in m.NODES if n != m.v0 
                for tp in m.PERIODS if tp <= t)
           )
        m.connectivityConstraint = Constraint(m.PERIODS, 
                                      rule=connectivityConstraint_rule, 
                                      doc="Faria2020 eq 10")
        
        m.obj = Objective(expr=sum(m.update_factor[t] 
                           * m.prize[n] 
                           * (1 - m.YT[n, t]) 
                           for t in m.PERIODS 
                           for n in m.NODES
                          )
                 + sum(m.edge_weight[e,t] * m.XT[e,t] 
                       for t in m.PERIODS 
                       for e in m.EDGES), sense=minimize,
                  doc="Faria2020 MPPCST eq 4"
                 )
        
        self.m = m
        
    def solve(self, *args, **kwargs):
        solver = SolverFactory("cbc", executable=SOLVER_DIR / "cbc")
        if "tee" not in kwargs:
            kwargs["tee"] = self.verbose
        results = solver.solve(self.m, *args, **kwargs)
        return results
    
    @staticmethod
    def print_results(results: dict):
        """Pretty print the results (or any dictionary)"""
        def pretty(d, indent=0):
           for key, value in d.items():
              print('\t' * indent + str(key))
              if isinstance(value, dict):
                 pretty(value, indent+1)
              else:
                 print('\t' * (indent+1) + str(value))
                    
        pretty(results)

In [6]:
stp = stparser.SteinerTreeProblem(stp_file="data/raw/ES10FST/es10fst02.stp", 
                                  update_factors=[0.5, 1.0], 
                                  default_prize=1.0)

In [7]:
mppcst = MPPCST(stp, v0=1, update_factor={1: 1.0, 2: 0.8}, distance_limit={1: 20.0, 2: 15})

In [8]:
results = mppcst.solve()

In [9]:
mppcst.m.display()

Model unknown

  Variables:
    YT : Size=28, Index=YT_index
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         (1, 1) :     0 :   1.0 :     1 : False : False : Binary
         (1, 2) :     0 :   1.0 :     1 : False : False : Binary
         (2, 1) :     0 :   0.0 :     1 : False : False : Binary
         (2, 2) :     0 :   0.0 :     1 : False : False : Binary
         (3, 1) :     0 :   0.0 :     1 : False : False : Binary
         (3, 2) :     0 :   0.0 :     1 : False : False : Binary
         (4, 1) :     0 :   0.0 :     1 : False : False : Binary
         (4, 2) :     0 :   0.0 :     1 : False : False : Binary
         (5, 1) :     0 :   0.0 :     1 : False : False : Binary
         (5, 2) :     0 :   0.0 :     1 : False : False : Binary
         (6, 1) :     0 :   0.0 :     1 : False : False : Binary
         (6, 2) :     0 :   0.0 :     1 : False : False : Binary
         (7, 1) :     0 :   0.0 :     1 : False : False : Binary
         (7, 2) :     0 :   0

In [10]:
MPPCST.print_results(results)

Problem
	
- Name: unknown
  Lower bound: 23.4
  Upper bound: 23.4
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 52
  Number of binary variables: 54
  Number of integer variables: 54
  Number of nonzeros: 52
  Sense: minimize

Solver
	
- Status: ok
  User time: -1.0
  System time: 0.06
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.04965686798095703

Solution
	
- number of solutions: 0
  number of solutions displayed: 0



# Read/Write GraphML

In [11]:
stp.write_graphml("test_graph.graphml")

In [12]:
gml = nx.read_graphml("test_graph.graphml")

In [13]:
gml.nodes(data=True)

NodeDataView({'1': {'prize': 1.0, 'is_terminal': 1, 'x': 1470158, 'y': 6131368}, '2': {'prize': 1.0, 'is_terminal': 1, 'x': 1251936, 'y': 2125058}, '3': {'prize': 1.0, 'is_terminal': 1, 'x': 7411042, 'y': 2036110}, '4': {'prize': 1.0, 'is_terminal': 1, 'x': 8044316, 'y': 4252316}, '5': {'prize': 1.0, 'is_terminal': 1, 'x': 2116787, 'y': 2721307}, '6': {'prize': 1.0, 'is_terminal': 1, 'x': 6624317, 'y': 7419167}, '7': {'prize': 1.0, 'is_terminal': 1, 'x': 8043960, 'y': 5698598}, '8': {'prize': 1.0, 'is_terminal': 1, 'x': 6684968, 'y': 2824909}, '9': {'prize': 1.0, 'is_terminal': 1, 'x': 4043257, 'y': 1895910}, '10': {'prize': 1.0, 'is_terminal': 1, 'x': 4978816, 'y': 2117592}, '11': {'prize': 1.0, 'is_terminal': 0, 'x': 4043257, 'y': 2117592}, '12': {'prize': 1.0, 'is_terminal': 0, 'x': 1470158, 'y': 2721307}, '13': {'prize': 1.0, 'is_terminal': 0, 'x': 7411042, 'y': 2824909}, '14': {'prize': 1.0, 'is_terminal': 0, 'x': 8043960, 'y': 4252316}})

# draft

In [14]:
stp.graph_attributes()

{'periods': 2, 'update_factor-01': 0.5, 'update_factor-02': 1.0}

In [15]:
def refactor_attribute_periods(self,
                               attribute_list=["update_factor"],
                               stp_attributes_dict=None,
                               attribute_format=None,
                               ):
    """Refactor multi-period attributes into one list-valued attribute.
    
    The list order corresponds to the values for different periods.
    """
    if stp_attributes_dict is None:
        stp_attributes_dict=self.graph_attributes()
    if attribute_format is None:
        attribute_format = self.graph_attribute_format
        
    updated_attributes = dict.fromkeys(attribute_list, 
                                       [None] * stp.num_periods()
                                       )
    for attribute in attribute_list:
        attribute_value_list = []
        for t in stp.iter_periods():
            attribute_name = (attribute_format.format(attribute) 
                              + stp.period_suffix_format.format(t))
            attribute_value_list.append(stp_attributes_dict.get(attribute_name, None))
        updated_attributes[attribute] = attribute_value_list
    return updated_attributes

In [16]:
stp.refactor_attribute_periods()

{'update_factor': [0.5, 1.0]}